## 9. Testing and Debugging

### 76 Verify Related Behaviors in `TestCase` Subclasses

```python
# utils.py

def to_str(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return data.decode('utf-8')
    else:
        raise TypeError('Must supply str or bytes, '
                        'found: %r' % data)
```

```python
# utils_test.py

from unittest import TestCase, main
from utils import to_str

class UtilsTestCase(TestCase):
    def test_to_str_bytes(self):
        self.assertEqual('hello', to_str(b'hello'))

    def test_to_str_str(self):
        self.assertEqual('hello', to_str('hello'))

    def test_failing(self):
        self.assertEqual('incorrect', to_str('hello'))

if __name__ == '__main__':
    main()
```

```shell
$ python utils_test.py 
F..
======================================================================
FAIL: test_failing (__main__.UtilsTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "utils_test.py", line 28, in test_failing
    self.assertEqual('incorrect', to_str('hello'))
AssertionError: 'incorrect' != 'hello'
- incorrect
+ hello


----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=1)
```

```shell
$ python utils_test.py UtilsTestCase.test_to_str_bytes 
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK
```

```python
# assert_test.py

from unittest import TestCase, main
from utils import to_str

class AssertTestCase(TestCase):
    def test_assert_helper(self):
        expected = 12
        found = 2 * 5
        self.assertEqual(expected, found)

    def test_assert_statement(self):
        expected = 12
        found = 2 * 5
        assert expected == found

if __name__ == '__main__':
    main()
```

```shell
$ python assert_test.py
FF
======================================================================
FAIL: test_assert_helper (__main__.AssertTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "assert_test.py", line 24, in test_assert_helper
    self.assertEqual(expected, found)
AssertionError: 12 != 10

======================================================================
FAIL: test_assert_statement (__main__.AssertTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "assert_test.py", line 29, in test_assert_statement
    assert expected == found
AssertionError

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=2)
```

```python
# utils_error_test.py

from unittest import TestCase, main
from utils import to_str

class UtilsErrorTestCase(TestCase):
    def test_to_str_bad(self):
        with self.assertRaises(TypeError):
            to_str(object())

    def test_to_str_bad_encoding(self):
        with self.assertRaises(UnicodeDecodeError):
            to_str(b'\xfa\xfa')

if __name__ == '__main__':
    main()
```

```shell
$ python utils_error_test.py
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK
```

```python
# helper_test.py

from unittest import TestCase, main

def sum_squares(values):
    cumulative = 0
    for value in values:
        cumulative += value ** 2
        yield cumulative

class HelperTestCase(TestCase):
    def verify_complex_case(self, values, expected):
        expect_it = iter(expected)
        found_it = iter(sum_squares(values))
        test_it = zip(expect_it, found_it)

        for i, (expect, found) in enumerate(test_it):
            self.assertEqual(
                expect,
                found,
                f'Index {i} is wrong')

        # Verify both generators are exhausted
        try:
            next(expect_it)
        except StopIteration:
            pass
        else:
            self.fail('Expected longer than found')

        try:
            next(found_it)
        except StopIteration:
            pass
        else:
            self.fail('Found longer than expected')

    def test_wrong_lengths(self):
        values = [1.1, 2.2, 3.3]
        expected = [
            1.1**2,
        ]
        self.verify_complex_case(values, expected)

    def test_wrong_results(self):
        values = [1.1, 2.2, 3.3]
        expected = [
            1.1**2,
            1.1**2 + 2.2**2,
            1.1**2 + 2.2**2 + 3.3**2 + 4.4**2,
        ]
        self.verify_complex_case(values, expected)

if __name__ == '__main__':
    main()
```

```shell
$ python helper_test.py 
FF
======================================================================
FAIL: test_wrong_lengths (__main__.HelperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "helper_test.py", line 57, in test_wrong_lengths
    self.verify_complex_case(values, expected)
  File "helper_test.py", line 50, in verify_complex_case
    self.fail('Found longer than expected')
AssertionError: Found longer than expected

======================================================================
FAIL: test_wrong_results (__main__.HelperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "helper_test.py", line 66, in test_wrong_results
    self.verify_complex_case(values, expected)
  File "helper_test.py", line 32, in verify_complex_case
    self.assertEqual(
AssertionError: 36.3 != 16.939999999999998 : Index 2 is wrong

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=2)
```

```python
# data_driven_test.py

from unittest import TestCase, main
from utils import to_str

class DataDrivenTestCase(TestCase):
    def test_good(self):
        good_cases = [
            (b'my bytes', 'my bytes'),
            ('no error', b'no error'),  # This one will fail
            ('other str', 'other str'),
        ]
        for value, expected in good_cases:
            with self.subTest(value):
                self.assertEqual(expected, to_str(value))

    def test_bad(self):
        bad_cases = [
            (object(), TypeError),
            (b'\xfa\xfa', UnicodeDecodeError),
        ]
        for value, exception in bad_cases:
            with self.subTest(value):
                with self.assertRaises(exception):
                    to_str(value)

if __name__ == '__main__':
    main()
```

```shell
$ python data_driven_test.py
.
======================================================================
FAIL: test_good (__main__.DataDrivenTestCase) [no error]
----------------------------------------------------------------------
Traceback (most recent call last):
  File "data_driven_test.py", line 29, in test_good
    self.assertEqual(expected, to_str(value))
AssertionError: b'no error' != 'no error'

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=1)
```

> - `unittest` 내장 모듈에 있는 `TestCase` 클래스의 하위 클래스를 정의하고 테스트하려는 동작마다 메서드를 정의함으로써 테스트를 정의할 수 있다. `TestCase` 하위 클래스 안에서 테스트 메서드의 이름은 test로 시작해야 한다.
> - 테스트 안에서는 파이썬 내장 `assert` 문을 사용하지 말고, `assertEqual`과 같이 `TestCase` 클래스에 정의된 여러 가지 헬퍼 메서드를 사용해 원하는 동작을 확인한다.
> - 준비 코드를 줄이려면 `subTest` 헬퍼 메서드를 사용해 데이터 기반 테스트를 정의하라.